In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta

In [7]:
import ensembleReader as er

In [11]:
reload(er)
dataReader = er.EdfDatasetEnsembler("dev_test", "01_tcp_ar",)

In [13]:
dataReader[0].shape

(500, 21)

In [14]:
len(dataReader)

17684

In [17]:
dataReader[len(dataReader) - 1]

array([[ -8.20419609,   1.67389091,  -5.08097727, ...,  -6.46100384,
         -1.15879583,  -7.11470127],
       [ -9.88747804,   6.41976591,  -8.43739631, ...,  -4.98027789,
          0.53376398,  -2.58960629],
       [ -8.29999841,   6.97976468, -11.63903722, ...,  -0.54031085,
         -0.85139844,  -5.96522476],
       ...,
       [ 15.19887989,  -0.93114334,  10.4655374 , ...,   9.99249828,
         29.96026345, -10.51160814],
       [ 16.99928043,   2.44134432,  10.78463447, ...,  10.63653747,
         31.29970288,  -9.44407936],
       [ 18.0702209 ,   5.99205425,  18.85250359, ...,  12.6842008 ,
         34.04422281,  -1.23419783]])

In [26]:
genderFiles = cta.demux_to_tokens(cta.getGenderAndFileNames("dev_test", "01_tcp_ar", convert_gender_to_num=True))

In [28]:
samplingInfo = pkl.load(open("../test_standardized_edf_ensemble_sample_info.pkl","rb"))

In [40]:
reload(er)
ale = er.AdditionalLabelEndpoints("combined", "01_tcp_ar", sampleInfo=samplingInfo)

In [41]:
gends = ale.get_genders()

KeyError: '/home/ms994/v1.5.0/edf/combined/01_tcp_ar/000/00007475/s003_2013_03_22/00007475_s003_t000.edf'

In [2]:
import keras
import pickle as pkl

Using TensorFlow backend.


In [3]:
testData = pkl.load(open("../standardized_combined_simple_ensemble_test_data.pkl", 'rb'))
def generate_x_y(data):
    x_data = np.stack([datum[0] for datum in data])
    x_data = x_data.reshape((*x_data.shape, 1))
    x_data.transpose(0, 2, 1, 3)
    y_data = np.array([datum[1] for datum in data])
    y_data = keras.utils.to_categorical(y_data)
    return x_data, y_data
testDataX, testDataY = generate_x_y(testData)
del testData

In [5]:
model = keras.models.load_model("yolo_bin_acc.h5")

In [6]:
y_pred = model.predict(testDataX)

NameError: name 'roc_auc_score' is not defined

In [7]:
from sklearn.metrics import roc_auc_score

In [8]:
roc_auc_score(testDataY.argmax(axis=1), y_pred.argmax(axis=1))

0.5955394210543571

In [9]:
samplingInfo = pkl.load(open("../test_standardized_edf_ensemble_sample_info.pkl","rb"))

In [ ]:
reload(read)

In [12]:
edf_tokens = list(set([samplingInfo[key]["token_file_path"] for key in samplingInfo.keys()]))

In [13]:
import ensembleReader as er
ensembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", generate_sample_info=False, edf_tokens=edf_tokens)

In [ ]:
len(edf_tokens)

In [14]:
ensembler.sampleInfo = samplingInfo
ensembler.labels = testDataY.argmax(axis=1)
ensembler.edf_tokens = edf_tokens

In [ ]:
from addict import Dict

In [ ]:
pred_vs_true = Dict()

In [19]:
trueLabel, pred = ensembler.getEnsemblePrediction(y_pred, mode=er.EdfDatasetEnsembler.ENSEMBLE_PREDICTION_EQUAL_VOTE)

In [20]:
roc_auc_score(trueLabel, pred)

0.6519588707410763